https://github.com/benalexkeen/Introduction-to-linear-programming/

In [1]:
%load_ext nb_black

from pulp import LpProblem, LpStatus, LpVariable, lpSum, value
from pulp import LpBinary, LpMinimize, LpMaximize

<IPython.core.display.Javascript object>

In [2]:
prob = LpProblem("Making Sausage", LpMinimize)

sausage_types = ["economy", "premium"]

ingredients = ["pork", "wheat", "starch"]

/usr/local/lib/python3.7/site-packages/pulp/pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


<IPython.core.display.Javascript object>

In [3]:
weight = LpVariable.dicts(
    "weight kg",
    ((sausage, ingredient) for sausage in sausage_types for ingredient in ingredients),
    lowBound=0,
    cat="Continuous",
)

<IPython.core.display.Javascript object>

In [4]:
# Objective Function
prob += lpSum(
    [
        4.32 * weight[(sausage, "pork")]
        + 2.46 * weight[(sausage, "wheat")]
        + 1.86 * weight[(sausage, "starch")]
        for sausage in sausage_types
    ]
)

<IPython.core.display.Javascript object>

In [5]:
# Constraints

# 350 economy and 500 premium sausages at 0.05 kg
prob += lpSum([weight["economy", i] for i in ingredients]) == 350 * 0.05
prob += lpSum([weight["premium", i] for i in ingredients]) == 500 * 0.05

# Economy has >= 40% pork, premium >= 60% pork
prob += weight["economy", "pork"] >= (
    0.4 * lpSum([weight["economy", i] for i in ingredients])
)

prob += weight["premium", "pork"] >= (
    0.6 * lpSum([weight["premium", i] for i in ingredients])
)

# Sausages must be <= 25% starch
prob += weight["economy", "starch"] <= (
    0.25 * lpSum([weight["economy", i] for i in ingredients])
)

prob += weight["premium", "starch"] <= (
    0.25 * lpSum([weight["premium", i] for i in ingredients])
)

# We have at most 30 kg of pork, 20 kg of wheat and 17 kg of starch available
prob += lpSum([weight[s, "pork"] for s in sausage_types]) <= 30
prob += lpSum([weight[s, "wheat"] for s in sausage_types]) <= 20
prob += lpSum([weight[s, "starch"] for s in sausage_types]) <= 17

# We have at least 23 kg of pork to use up
prob += lpSum([weight[s, "pork"] for s in sausage_types]) >= 23

<IPython.core.display.Javascript object>

In [6]:
prob.solve()

LpStatus[prob.status]

'Optimal'

<IPython.core.display.Javascript object>

In [21]:
weight[("economy", "pork")].varValue

7.0

<IPython.core.display.Javascript object>

In [32]:
print(
    f"""
Economy
-------
    Pork:   {weight[("economy", "pork")].varValue:>5.2f} kg
    Starch: {weight[("economy", "starch")].varValue:>5.2f} kg
    Wheat:  {weight[("economy", "wheat")].varValue:>5.2f} kg
    
Premimum
--------
    Pork:   {weight[("premium", "pork")].varValue:>5.2f} kg
    Starch: {weight[("premium", "starch")].varValue:>5.2f} kg
    Wheat:  {weight[("premium", "wheat")].varValue:>5.2f} kg
    

Objective
=========
{value(prob.objective):>4.2f}
"""
)


Economy
-------
    Pork:    7.00 kg
    Starch:  4.38 kg
    Wheat:   6.12 kg
    
Premimum
--------
    Pork:   16.00 kg
    Starch:  6.25 kg
    Wheat:   2.75 kg
    

Objective
140.96



<IPython.core.display.Javascript object>

In [8]:
prob.variables

<bound method LpProblem.variables of Making_Sausage:
MINIMIZE
4.32*weight_kg_('economy',_'pork') + 1.86*weight_kg_('economy',_'starch') + 2.46*weight_kg_('economy',_'wheat') + 4.32*weight_kg_('premium',_'pork') + 1.86*weight_kg_('premium',_'starch') + 2.46*weight_kg_('premium',_'wheat') + 0.0
SUBJECT TO
_C1: weight_kg_('economy',_'pork') + weight_kg_('economy',_'starch')
 + weight_kg_('economy',_'wheat') = 17.5

_C2: weight_kg_('premium',_'pork') + weight_kg_('premium',_'starch')
 + weight_kg_('premium',_'wheat') = 25

_C3: 0.6 weight_kg_('economy',_'pork') - 0.4 weight_kg_('economy',_'starch')
 - 0.4 weight_kg_('economy',_'wheat') >= 0

_C4: 0.4 weight_kg_('premium',_'pork') - 0.6 weight_kg_('premium',_'starch')
 - 0.6 weight_kg_('premium',_'wheat') >= 0

_C5: - 0.25 weight_kg_('economy',_'pork')
 + 0.75 weight_kg_('economy',_'starch') - 0.25 weight_kg_('economy',_'wheat')
 <= 0

_C6: - 0.25 weight_kg_('premium',_'pork')
 + 0.75 weight_kg_('premium',_'starch') - 0.25 weight_kg_('premi

<IPython.core.display.Javascript object>